In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [5]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.2682,  0.0304, -0.1526]],

        [[-0.5370,  0.0346, -0.1958]],

        [[-0.3947,  0.0391, -0.1217]],

        [[-0.1854,  0.0740, -0.0979]],

        [[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>)
(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward>))


In [6]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [23]:
char_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        for char in word:
            if char not in char_to_ix:
                char_to_ix[char] = len(char_to_ix)
print(char_to_ix)

{'T': 0, 'h': 1, 'e': 2, 'd': 3, 'o': 4, 'g': 5, 'a': 6, 't': 7, 'p': 8, 'l': 9, 'E': 10, 'v': 11, 'r': 12, 'y': 13, 'b': 14, 'k': 15}


In [47]:
class CharLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(CharLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.char_embeddings = nn.Embedding(vocab_size, hidden_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
    
    def forward(self, word):
        embeds = self.char_embeddings(word)
        output, hidden = self.lstm(embeds.view(len(word), 1, -1))
        return hidden[0]

In [77]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim * 2, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, word, char_emb):
        embeds = self.word_embeddings(word)
        lstm_out, _ = self.lstm(torch.cat((embeds.view(1, 1, -1), char_emb), -1))
        tag_space = self.hidden2tag(lstm_out.view(1, -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores


In [78]:
class CharAndWordLSTM(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, char_vocab_size, word_vocab_size, tagset_size):
        super(CharAndWordLSTM, self).__init__()
        self.char_lstm = CharLSTM(embedding_dim, hidden_dim, char_vocab_size)
        self.lstm_tagger = LSTMTagger(embedding_dim, hidden_dim, word_vocab_size, tagset_size)
        
    def forward(self, chars, word):
        char_emb = self.char_lstm(chars)
        tag_scores = self.lstm_tagger(word, char_emb)
        return tag_scores

In [86]:
model = CharAndWordLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(char_to_ix), len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    char_level_inp = prepare_sequence(training_data[0][0][0], char_to_ix)
    word_level_inp = prepare_sequence(training_data[0][0], word_to_ix)
    tag_score = model(char_level_inp, word_level_inp[0])
    print(tag_score)

torch.Size([1, 3])
tensor([[-1.2349, -1.2361, -0.8708]])


In [90]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        tag_scores = []
        targets = prepare_sequence(tags, tag_to_ix)
        for word in sentence:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Step 2. Get our inputs ready for the network, that is, turn them into
            # Tensors of word indices.
            char_level_inp = prepare_sequence(word, char_to_ix)
            word_level_inp = prepare_sequence([word], word_to_ix)
            
            # Step 3. Run our forward pass.
            tag_score = model(char_level_inp, word_level_inp)
            tag_scores.append(tag_score)
        
        tag_scores = torch.squeeze(torch.stack(tag_scores))
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    char_level_inp = prepare_sequence(training_data[0][0][0], char_to_ix)
    word_level_inp = prepare_sequence(training_data[0][0], word_to_ix)
    tag_score = model(char_level_inp, word_level_inp[0])
    print(tag_score)

tensor([[-0.0361, -4.4324, -3.7471]])
